In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00


Simple Version

In [ ]:
from PIL import Image
import requests
import torch

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a giraffe"], images=image, return_tensors="pt", padding=True)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

In [ ]:
logits_per_image

tensor([[24.5701, 27.9732]])

In [ ]:
probs

tensor([[0.0322, 0.9678]])

Expanded Version

In [ ]:
from PIL import Image
import requests
import torch

from transformers import CLIPProcessor, CLIPModel

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])

In [ ]:
image_features = model.get_image_features(**inputs)

TypeError: ignored

In [ ]:
inputs['pixel_values'].shape, inputs['input_ids'].shape

(torch.Size([1, 3, 224, 224]), torch.Size([2, 7]))

In [ ]:
inputs['input_ids']

tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]])

In [ ]:
with torch.no_grad():
    vision_outputs = model.vision_model(
        pixel_values=inputs['pixel_values']
    )

    text_outputs = model.text_model(
        input_ids=inputs['input_ids']
    )

    image_embeds_ = vision_outputs[1]
    image_embeds = model.visual_projection(image_embeds_)

    text_embeds_ = text_outputs[1]
    text_embeds = model.text_projection(text_embeds_)

    # normalized features
    image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)
    text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)

    # cosine similarity as logits
    logit_scale = model.logit_scale.exp()
    logits_per_text = torch.matmul(text_embeds, image_embeds.t()) * logit_scale
    logits_per_image = logits_per_text.t()

In [ ]:
image_embeds_.shape, text_embeds_.shape

(torch.Size([1, 768]), torch.Size([2, 512]))

In [ ]:
image_embeds.shape, text_embeds.shape

(torch.Size([1, 512]), torch.Size([2, 512]))

In [ ]:
logits_per_text.shape, logits_per_image.shape

(torch.Size([2, 1]), torch.Size([1, 2]))

In [ ]:
probs = logits_per_image.softmax(dim=1)

In [ ]:
probs

tensor([[0.9949, 0.0051]])

The model should not differentiate between the order of positive pairs. In other words, (Image1, Text1) and (Text1, Image1) are the same positive pair. By using both logits_per_text and logits_per_image, we ensure that we handle both directions of positive pairs correctly.